In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from enum import Enum
from collections import namedtuple

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator

#### EV Charging network

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network_unbalanced(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(2)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

#### General Simulation setups

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

In [5]:
# charging network with evses
level2_ev_fleet_network = ev_fleet_level_2_network_unbalanced(transformer_cap=30)
level2_ev_fleet_network.magnitudes

24.096385542168676
43.47826086956522


array([43.47826087, 43.47826087, 43.47826087, 24.09638554, 24.09638554,
       24.09638554])

In [6]:
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

#### Generate synthetic events

In [ ]:
# Events with 32 EVs per weekday
sessions_30 = get_synth_events(30)

In [8]:
# 25 EV sessions per weekday
sessions_25 = get_synth_events(25)

In [10]:
# 20 EV sessions per weekday
sessions_20 = get_synth_events(20)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
# Checking session details
session_copy = deepcopy(sessions_20)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)


645 session_16 739 5.05481147384528 Plugin
651 session_19 771 13.503000045892565 Plugin
665 session_14 734 3.035231669599554 Plugin
656 session_8 711 2.858255979995037 Plugin
653 session_4 779 12.18907445041579 Plugin
683 session_2 800 14.930081658698578 Plugin
670 session_11 754 6.920124656859933 Plugin
660 session_3 773 12.00934834978649 Plugin
668 session_1 780 12.406536883839074 Plugin
665 session_0 784 0.820033725997049 Plugin
677 session_10 796 6.3612292687068805 Plugin
758 session_5 774 7.666548750587374 Plugin
722 session_12 762 7.199429255572878 Plugin
771 session_6 792 10.471508093052716 Plugin
764 session_13 802 8.937538906149191 Plugin
728 session_7 757 14.65366109666001 Plugin
666 session_15 770 8.118790757841735 Plugin
681 session_17 793 10.72414626769705 Plugin
668 session_18 783 13.08551079580028 Plugin
672 session_9 797 8.114246218089601 Plugin


In [12]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    # Start and End times are used when collecting data.
    # start = timezone.localize(datetime(2019, 6, 1))
    # end = timezone.localize(datetime(2019, 7, 1))

    start = timezone.localize(datetime(2023, 11, 22))
    end = timezone.localize(datetime(2023, 11, 23))

    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    # sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    sim = modified_simulator.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    
    # r['total_cost'] = r['energy_cost'] + r['demand_charge'] # original
    r['total_cost'] = r['energy_cost'] # modified
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    # return r, sim.charging_rates_as_df()
    return r

In [13]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)
rr = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

In [14]:
cost_min_obj = [modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-11)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=150, max_recompute=1)

#### Run Simulation

In [16]:
level2_30kW_untrl = run_experiment(level2_ev_fleet_network, uncontrolled, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_19', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_19', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_14', 'session_19', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_14', 'session_19', 'session_0', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_14', 'session_15', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging session

c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 653. Max violation is 11.937364972638854 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 654. Max violation is 11.937364972638854 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 655. Max violation is 11.937364972638854 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 656. Max violation is 11.93736497263886 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_sim

In [17]:
level2_30kW_llf = run_experiment(level2_ev_fleet_network, llf, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_8', 'session_4', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_8', 'session_3', 'session_4', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_8', 'session_3', 'session_4', 'session_19', 'session_14', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_8', 'session_3', 'session_0', 'session_4', 'session_19', 'session_14', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going ses

In [18]:
level2_30kW_cost_min = run_experiment(level2_ev_fleet_network, cost_min, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_8', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_3', 'session_8', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_14', 'session_3', 'session_8', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_0', 'session_14', 'session_3', 'session_8', 'session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going ses

In [19]:
level2_30kW_rr = run_experiment(level2_ev_fleet_network, rr, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_4', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_4', 'session_8', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_4', 'session_8', 'session_19', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_4', 'session_14', 'session_8', 'session_19', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_4', 'session_14', 'session_0', 'session_8', 'session_19', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going ses

In [17]:
ev_30 = pd.DataFrame({
    'Level 2: Unctrl: 30 kW : 7 EVSEs':  level2_30kW_untrl,
    'Level 2: LLF: 30 kW : 7 EVSEs': level2_30kW_llf,
    'Level 2: RR: 30 kW : 7 EVSEs': level2_30kW_rr,
    'Level 2: Min Cost: 30 kW : 7 EVSEs': level2_30kW_cost_min
})

In [18]:
ev_30

,Level 2: Unctrl: 30 kW : 7 EVSEs,Level 2: LLF: 30 kW : 7 EVSEs,Level 2: RR: 30 kW : 7 EVSEs,Level 2: Min Cost: 30 kW : 7 EVSEs
proportion_of_energy_delivered,1.000000,0.736997,0.696472,0.642248
energy_delivered,244.983067,180.551667,170.623750,157.339989
num_swaps,15.000000,13.000000,12.000000,9.000000
num_never_charged,0.000000,3.000000,4.000000,7.000000
energy_cost,17.036655,13.526201,12.666524,13.003088
demand_charge,1364.880000,453.822600,482.748750,476.312100
total_cost,17.036655,13.526201,12.666524,13.003088
$/kWh,0.069542,0.074916,0.074237,0.082643


In [20]:
ev_20_30 = pd.DataFrame({
    'Level 2: Unctrl: 30 kW : 7 EVSEs':  level2_30kW_untrl,
    'Level 2: LLF: 30 kW : 7 EVSEs': level2_30kW_llf,
    'Level 2: RR: 30 kW : 7 EVSEs': level2_30kW_rr,
    'Level 2: Min Cost: 30 kW : 7 EVSEs': level2_30kW_cost_min
})

In [21]:
ev_20_30

,Level 2: Unctrl: 30 kW : 7 EVSEs,Level 2: LLF: 30 kW : 7 EVSEs,Level 2: RR: 30 kW : 7 EVSEs,Level 2: Min Cost: 30 kW : 7 EVSEs
proportion_of_energy_delivered,1.000000,0.914458,0.913995,0.759910
energy_delivered,179.059108,163.742083,163.659167,136.068750
num_swaps,9.000000,8.000000,8.000000,9.000000
num_never_charged,0.000000,1.000000,1.000000,3.000000
energy_cost,13.152976,12.434926,12.432773,11.556102
demand_charge,870.299915,428.076000,482.748750,418.382250
total_cost,13.152976,12.434926,12.432773,11.556102
$/kWh,0.073456,0.075942,0.075967,0.084928


#### Simulation with 35 KW 7 EVSE system

In [22]:
# charging network with evses with 35 kW transformer
level2_ev_fleet_network_35 = ev_fleet_level_2_network_unbalanced(transformer_cap=35)
level2_ev_fleet_network_35.magnitudes

28.112449799196785
50.724637681159415


array([50.72463768, 50.72463768, 50.72463768, 28.1124498 , 28.1124498 ,
       28.1124498 ])

In [23]:
level2_35kW_untrl = run_experiment(level2_ev_fleet_network_35, uncontrolled, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4', 'session_14', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_0', 'session_19', 'session_4', 'session_14', 'session_3']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_14', 'session_3', 'session_15']
Waiting queue: OrderedDict()
High priority EV charging session

c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 653. Max violation is 4.6909881610446575 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 654. Max violation is 4.6909881610446575 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 655. Max violation is 4.6909881610446575 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:356: UserWarning: Invalid schedule provided at iteration 656. Max violation is 4.690988161044665 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_sim

In [24]:
level2_35kW_llf = run_experiment(level2_ev_fleet_network_35, llf, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_19', 'session_4']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_19', 'session_4', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_19', 'session_4', 'session_3', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_14', 'session_19', 'session_4', 'session_3', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_0', 'session_16', 'session_14', 'session_19', 'session_4', 'session_3', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going ses

In [25]:
level2_35kW_cost_min = run_experiment(level2_ev_fleet_network_35, cost_min, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_16', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_16', 'session_19']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_16', 'session_19', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_4', 'session_16', 'session_3', 'session_19', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_14', 'session_4', 'session_16', 'session_3', 'session_19', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_14', 'session_4', 'session_16', 'session_3', 'session_0', 'session_19', 'session_8']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going ses

In [26]:
level2_35kW_rr = run_experiment(level2_ev_fleet_network_35, rr, deepcopy(sessions_20))

on-going session: ['session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4', 'session_8', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_4', 'session_3', 'session_8', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_14', 'session_4', 'session_3', 'session_8', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going session: ['session_19', 'session_14', 'session_0', 'session_4', 'session_3', 'session_8', 'session_16']
Waiting queue: OrderedDict()
High priority EV charging sessions: []
on-going ses

In [24]:
ev_25_35 = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 7 EVSEs':  level2_35kW_untrl,
    'Level 2: LLF: 35 kW : 7 EVSEs': level2_35kW_llf,
    'Level 2: RR: 35 kW : 7 EVSEs': level2_35kW_rr,
    'Level 2: Min Cost: 35 kW : 7 EVSEs': level2_35kW_cost_min
})

In [25]:
ev_25_35

,Level 2: Unctrl: 35 kW : 7 EVSEs,Level 2: LLF: 35 kW : 7 EVSEs,Level 2: RR: 35 kW : 7 EVSEs,Level 2: Min Cost: 35 kW : 7 EVSEs
proportion_of_energy_delivered,1.000000,0.823026,0.772620,0.651218
energy_delivered,244.983067,201.627500,189.278750,159.537489
num_swaps,15.000000,14.000000,12.000000,9.000000
num_never_charged,0.000000,2.000000,4.000000,7.000000
energy_cost,17.036655,15.072156,14.167238,13.210623
demand_charge,1364.880000,505.315800,559.988550,531.062400
total_cost,17.036655,15.072156,14.167238,13.210623
$/kWh,0.069542,0.074752,0.074849,0.082806


In [27]:
ev_20_35 = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 7 EVSEs':  level2_35kW_untrl,
    'Level 2: LLF: 35 kW : 7 EVSEs': level2_35kW_llf,
    'Level 2: RR: 35 kW : 7 EVSEs': level2_35kW_rr,
    'Level 2: Min Cost: 35 kW : 7 EVSEs': level2_35kW_cost_min
})

In [28]:
ev_20_35

,Level 2: Unctrl: 35 kW : 7 EVSEs,Level 2: LLF: 35 kW : 7 EVSEs,Level 2: RR: 35 kW : 7 EVSEs,Level 2: Min Cost: 35 kW : 7 EVSEs
proportion_of_energy_delivered,1.000000,0.915433,0.912995,0.811094
energy_delivered,179.059108,163.916667,163.480000,145.233750
num_swaps,8.000000,8.000000,8.000000,9.000000
num_never_charged,0.000000,1.000000,1.000000,3.000000
energy_cost,13.152976,12.509469,12.430685,12.355436
demand_charge,870.299915,486.005850,543.935700,457.002150
total_cost,13.152976,12.509469,12.430685,12.355436
$/kWh,0.073456,0.076316,0.076038,0.085073


In [ ]:
# offline

# Create an instance of the class
# algo = modified_adacharge.AdaptiveChargingAlgorithmOffline(cost_min, constraint_type="SOC", enforce_energy_equality=False, solver="MOSEK", peak_limit=150, verbose=False)

# # Register interface
# algo.register_interface(level2_ev_fleet_network_35)

# # Register events
# algo.register_events(sessions_20)

# # Perform the optimization
# algo.solve()

# Get the charging schedule for the active EVs
# schedule = algo.schedule(active_evs)